<a href="https://colab.research.google.com/github/viraatdas/Purdue_CompVis_Workshop/blob/1_import-libraries/speed_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("wus good")

wus good


Import video files

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Import libraries

In [0]:
import os
import csv
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt